In [ ]:
%pylab inline

pylab.rcParams['figure.figsize'] = (6, 6)

import math
import cmath          # math functions for complex numbers
import numpy as np
import matplotlib.pyplot as plt

import scipy
import scipy.stats
import pandas as pd

import ipywidgets
from ipywidgets import interact

**TODO**:
* outils:
    * charger fichier fits benchmark
    * charger fichier fits mrfilter
    * appeler mr_filter / mr_transform, afficher l'entrée, la sortie et les plans (+ histogrammes)
    * la même chose que la GUI
* charge loss
* threshold
* spellcheck

## Introduction

###  Context

On travaille sur des images provenant de simulations montecarlo de telescopes gamma (~100 GeV to ~300 TeV)

Exemple:
<img src="./figs/image31.gif" width="50%"></img>

Processus:
1. nettoyer l'image (suppression du bruit electronique et du NSB)
2. déterminer l'axe de la gerbe gamma sur l'image
3. par stereoscopie determiner l'origine de la gerbe

|                                                  |                                                  |
| ------------------------------------------------ | ------------------------------------------------ |
| <img src="./figs/image30.png" width="90%"></img> | <img src="./figs/image32.png" width="90%"></img> |
| <img src="./figs/image33.png" width="90%"></img> | <img src="./figs/Arrayscheme2_x.png" width="90%"></img> |


### Image types

#### FlashCam telescopes: 1764 (hexagonal) pixels

<img src="./figs/geom_3_flashcam.png" width="40%">

#### ASTRI telescopes: 2368 (rectangular) pixels

<img src="./figs/geom_4_astri.png" width="40%">

#### and others...

...

### Images I use so far

- Cropped images from Astri telescopes => 40x40 pixels
- Set of ~15000 images

<img src="./figs/geom_4_astri_crop3.svg" width="60%">

### Cleaning algorithm reference

The "Tailcut clean" algorithm:
- Keep pixels above a given threshold (10 PE)
- Keep some neighbors of these selected pixels: those above a second (lower) threshold (5 PE)

Goal:
- use sparse2d tools to get a better image cleaning (electronic noise and NSB removal)...
- to improve event reconstruction..
- and thus improve observatory's sensibility

## Some stats

### Total number of photoelectrons (NPE) per image

In [ ]:
%run ./utils/plot_metadata_histogram.py --key=npe --logx --title="Number of photoelectrons (NPE) per image" ./xps/2017_02_04/score_gamma_all_null_ref.json

### Noise histogram (telescope 1 only)

In [ ]:
%run ./utils/plot_noise_histogram.py --notebook --logy --min-npe=50 --max-npe=1500 ./testset/gamma/tel1/

abs(noise) $\sim \text{Poisson}(\lambda \approx 2)$

### Noise histogram (telescope 2 only)

In [ ]:
%run ./utils/plot_noise_histogram.py --notebook --logy --min-npe=50 --max-npe=1500 ./testset/gamma/tel2/

## Questions

### Remove isolated pixels after cleaning: l'option "-k" ne semble pas avoir d'effet

<img src="./figs/kill_isolated_pixels/ev458109_tel1.png" width="75%">

<img src="./figs/kill_isolated_pixels/ev481109_tel1_log.png" width="75%">

### Charge loss

**TODO...**

### Threshold WT

* hard filter do 1, 0.6, ...
* on voudrait faire le contraire: mettre un seuil plus bas pour les petites échelles (filtrer moins) et plus haut pour les grandes échelles (filtrer plus) pour conserver plus de détails (nos objets ne font que quelques pixels)

### Hexagonal images and corners

**Objectif n°1**: gérer les images ayant des "trous" dans les coins

Idée:
- pouvoir activer prématurement le "mirroring" d'un tuple de pixels (i.e. ce qui se passe quand on atteind le bord de l'image dans smooth_bspline) en mettant des pixels à une valeur particulière (FLOAT_MIN, NaN, ...)

<img src="./figs/geom_4_astri.png" width="40%">

**Objectif n°2**: travailler sur des images faites d'une grille de pixels hexagonaux

<img src="./figs/geom_3_flashcam.png" width="40%">

Idée:
    - ajouter une itération dans smooth_bspline: applique la convolution suivant un 3e axe
    - suppose de changer la structure de donnée des images (workaround de travailler avec des cubes)
    - l'idée n'est pas de changer la structure de donnée des images à un niveau global (imutilement long et compliqué) mais plutôt très localement pour une configuration particulière de filtre

<img src="./figs/hexagonal_grid.png" width="25%">

In [ ]:
# %load /Users/jdecock/bin/isap/cxx/sparse2d/src/libsparse2d/IM_Smooth.cc

void smooth_bspline (const Iint & Im_in,
                     Iint &Im_out,
                     type_border Type,
                     int Step_trou)
{
    int Nl = Im_in.nl();
    int Nc = Im_in.nc();
    int i,j,Step;
    float Coeff_h0 = 3. / 8.;
    float Coeff_h1 = 1. / 4.;
    float Coeff_h2 = 1. / 16.;
    Ifloat Buff(Nl,Nc,"Buff smooth_bspline");

    Step = (int)(pow((double)2., (double) Step_trou) + 0.5);

    for (i = 0; i < Nl; i ++)
    for (j = 0; j < Nc; j ++)
       Buff(i,j) = Coeff_h0 * (float)  Im_in(i,j)
                 + Coeff_h1 * (float) (  Im_in (i, j-Step, Type) 
                                       + Im_in (i, j+Step, Type)) 
                 + Coeff_h2 * (float) (  Im_in (i, j-2*Step, Type) 
                                       + Im_in (i, j+2*Step, Type));

    for (i = 0; i < Nl; i ++)
    for (j = 0; j < Nc; j ++)
       Im_out(i,j) = (int)(Coeff_h0 * Buff(i,j) +
                 + Coeff_h1 * (float) (  Buff (i-Step, j, Type) 
                                       + Buff (i+Step, j, Type)) 
                 + Coeff_h2 * (float) (  Buff (i-2*Step, j, Type) 
                                       + Buff (i+2*Step, j, Type)) + 0.5);

}

### Wrapper

Pour faire du filtrage en temps réel, l'utilisation de fichiers pour échanger les données avec mr_filter/mr_transform/... pose problème (temps d'accès aux fichiers trop long).

Idéalement il nous faudrait des wrappers Python.

Sinon, une amélioration possible peu couteuse en temps de developpement serait d'échanger les données via stdin/stdout.